# run dirs ws-out

In [1]:
! for i in `cat ../Orchestration/junk` ; do echo $i; aws s3 ls ws-out/CONUS/$i ;echo '---------------' ; done


Run02_17_2022/
                           PRE r50t7/
---------------
Run02_24_2022/
                           PRE conus_r50t6/
---------------
Run02_25_2022/
                           PRE conus_r50t5/
                           PRE conus_r50t8/
---------------
Run03_01_2022/
                           PRE conus_r37t0/
---------------
Run03_03_2022/
                           PRE conus_r37t1/
                           PRE conus_r37t2/
---------------
Run03_07_2022/
                           PRE conus_r37t3/
---------------
Run03_11_2022/
                           PRE conus_r50t9/
---------------


# output dirs 

In [2]:
! aws s3 ls ws-enduser/CONUS/

                           PRE r37.0_tile0/
                           PRE r37.0_tile1/
                           PRE r37.0_tile2/
                           PRE r37.0_tile3/
                           PRE r50.0_tile5/
                           PRE r50.0_tile6/
                           PRE r50.0_tile7/
                           PRE r50.0_tile8/
                           PRE r50.0_tile9/


In [3]:
import boto3

In [4]:
def s3_bucket_analyze(bucket, prefix):

    objs = []

    print("bucket", bucket)
    print("prefix", prefix)

    bucket_name = bucket
    prefix = prefix

    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    cnt=0
    storage_class_h = {'STANDARD' : 0,
                   'GLACIER' : 0,
                   'INTELLIGENT_TIERING' : 0,
                  }
    sum_class_h = {'STANDARD' : 0,
                   'GLACIER' : 0,
                   'INTELLIGENT_TIERING' : 0,
                  }
    sum = 0
    for obj in bucket.objects.filter(Prefix=prefix):
        storage_class_h[obj.storage_class] = storage_class_h[obj.storage_class] + 1
        cnt = cnt + 1
        if not cnt%1000:
            print(bucket, "bucket object count = ", cnt, flush=True)
        sum = sum + obj.size
        sum_class_h[obj.storage_class] = sum_class_h[obj.storage_class] + obj.size
        my_obj = {
                'bucket_name':obj.bucket_name,
                'key':obj.key,
                'size':obj.size,
                'class':obj.storage_class
                }
        objs.append(my_obj)

    print ("COUNT=", cnt)
    for ky in storage_class_h.keys():
        print(ky, storage_class_h[ky])
        sum = sum_class_h[ky]
        print(ky, sum_class_h[ky])
        gig = sum/(1024*1024*1024)
        print (ky, "GBYTES=", gig)
        if ky == 'GLACIER': 
            cost=.007
        else: 
            cost=.023
        print (ky, "Cost/Month=", gig * cost)
        print ("----" * 25)
    print("END LOOP")
    my_key = 'STANDARD'
    ret_gbytes = sum_class_h[my_key]/(1024*1024*1024)
    if (ret_gbytes < 1):
        ret_gbytes = 1
    ret_costs = .023 * ret_gbytes
    print('G:', ret_gbytes, ret_costs)
    return objs

In [5]:
bucket='ws-enduser'
my_prefix='CONUS/'

In [6]:
objs = s3_bucket_analyze(bucket, my_prefix)


bucket ws-enduser
prefix CONUS/
s3.Bucket(name='ws-enduser') bucket object count =  1000
s3.Bucket(name='ws-enduser') bucket object count =  2000
s3.Bucket(name='ws-enduser') bucket object count =  3000
s3.Bucket(name='ws-enduser') bucket object count =  4000
s3.Bucket(name='ws-enduser') bucket object count =  5000
s3.Bucket(name='ws-enduser') bucket object count =  6000
s3.Bucket(name='ws-enduser') bucket object count =  7000
s3.Bucket(name='ws-enduser') bucket object count =  8000
s3.Bucket(name='ws-enduser') bucket object count =  9000
s3.Bucket(name='ws-enduser') bucket object count =  10000
s3.Bucket(name='ws-enduser') bucket object count =  11000
s3.Bucket(name='ws-enduser') bucket object count =  12000
s3.Bucket(name='ws-enduser') bucket object count =  13000
s3.Bucket(name='ws-enduser') bucket object count =  14000
s3.Bucket(name='ws-enduser') bucket object count =  15000
s3.Bucket(name='ws-enduser') bucket object count =  16000
s3.Bucket(name='ws-enduser') bucket object count 

s3.Bucket(name='ws-enduser') bucket object count =  142000
s3.Bucket(name='ws-enduser') bucket object count =  143000
s3.Bucket(name='ws-enduser') bucket object count =  144000
s3.Bucket(name='ws-enduser') bucket object count =  145000
s3.Bucket(name='ws-enduser') bucket object count =  146000
s3.Bucket(name='ws-enduser') bucket object count =  147000
s3.Bucket(name='ws-enduser') bucket object count =  148000
s3.Bucket(name='ws-enduser') bucket object count =  149000
s3.Bucket(name='ws-enduser') bucket object count =  150000
s3.Bucket(name='ws-enduser') bucket object count =  151000
s3.Bucket(name='ws-enduser') bucket object count =  152000
s3.Bucket(name='ws-enduser') bucket object count =  153000
s3.Bucket(name='ws-enduser') bucket object count =  154000
s3.Bucket(name='ws-enduser') bucket object count =  155000
s3.Bucket(name='ws-enduser') bucket object count =  156000
s3.Bucket(name='ws-enduser') bucket object count =  157000
s3.Bucket(name='ws-enduser') bucket object count =  1580

In [7]:
objs[0]

{'bucket_name': 'ws-enduser',
 'key': 'CONUS/r37.0_tile0/1999/dd_199901.tif',
 'size': 6063364,
 'class': 'STANDARD'}

In [8]:
new_objs=[]
for o in objs:
    a = o['key'].split('/')
    project = a[0]
    tile = a[1]
    year = a[2]
    basenm = a[3]
    new_o = {
        'bucket_name': o['bucket_name'],
        'project': project,
        'tile': tile,
        'year': year,
        'basename': basenm,
        'size': o['size']
    }
    new_objs.append(new_o)

In [9]:
import pandas as pd

df = pd.DataFrame(new_objs)

In [10]:
df

,bucket_name,project,tile,year,basename,size
0,ws-enduser,CONUS,r37.0_tile0,1999,dd_199901.tif,6063364
1,ws-enduser,CONUS,r37.0_tile0,1999,dd_199902.tif,10258280
2,ws-enduser,CONUS,r37.0_tile0,1999,dd_199903.tif,5193049
3,ws-enduser,CONUS,r37.0_tile0,1999,dd_199904.tif,7873466
4,ws-enduser,CONUS,r37.0_tile0,1999,dd_199905.tif,2429679
...,...,...,...,...,...,...
222693,ws-enduser,CONUS,r50.0_tile9,2020,srf_202008.tif,1295984
222694,ws-enduser,CONUS,r50.0_tile9,2020,srf_202009.tif,1108483
222695,ws-enduser,CONUS,r50.0_tile9,2020,srf_202010.tif,10106845
222696,ws-enduser,CONUS,r50.0_tile9,2020,srf_202011.tif,16757676


In [11]:
tiles = df['tile'].unique()

In [12]:
tiles

array(['r37.0_tile0', 'r37.0_tile1', 'r37.0_tile2', 'r37.0_tile3',
       'r50.0_tile5', 'r50.0_tile6', 'r50.0_tile7', 'r50.0_tile8',
       'r50.0_tile9'], dtype=object)

In [13]:
tdf = df[df['tile'] == tiles[0]]

In [14]:
tdf

,bucket_name,project,tile,year,basename,size
0,ws-enduser,CONUS,r37.0_tile0,1999,dd_199901.tif,6063364
1,ws-enduser,CONUS,r37.0_tile0,1999,dd_199902.tif,10258280
2,ws-enduser,CONUS,r37.0_tile0,1999,dd_199903.tif,5193049
3,ws-enduser,CONUS,r37.0_tile0,1999,dd_199904.tif,7873466
4,ws-enduser,CONUS,r37.0_tile0,1999,dd_199905.tif,2429679
...,...,...,...,...,...,...
34760,ws-enduser,CONUS,r37.0_tile0,2020,srf_2020089.tif,1707382
34761,ws-enduser,CONUS,r37.0_tile0,2020,srf_202009.tif,839751
34762,ws-enduser,CONUS,r37.0_tile0,2020,srf_202010.tif,826999
34763,ws-enduser,CONUS,r37.0_tile0,2020,srf_202011.tif,842814


In [15]:
tdf.describe()

,size
count,3.476500e+04
mean,5.233470e+07
std,3.151742e+07
min,8.269600e+05
25%,8.832906e+06
50%,7.388930e+07
75%,7.448758e+07
max,7.681894e+07


In [16]:
variables = ['etasw','srf', 'dd','netet', 'etc']

In [17]:
import re
def validate_monthly_data(tdf, tile):
    total_work_left = 0
    for variable in variables:
        for year in range(2000,2020+1):
            ytdf = tdf[tdf['year'] == str(year)]
            basenames = ytdf['basename'].to_list()
            #print(basenames)
            #print(ytdf)
            #for m in range(1,12+1):
            rematch=f'{variable}_{year}\d\d.tif'
            r = re.compile(rematch)
            my_monthly_list_by_var = [x for x in basenames if r.match(x)]
            my_len = len(my_monthly_list_by_var)
            if my_len < 12:
                total_work_left = total_work_left + 12 - my_len
                print(tile, variable, year, len(my_monthly_list_by_var))
    return total_work_left

            

In [18]:
for tile in tiles:
    tdf = df[df['tile'] == tile]
    total_work_left = validate_monthly_data(tdf, tile)
    print(tile, total_work_left)

r37.0_tile0 0
r37.0_tile1 0
r37.0_tile2 0
r37.0_tile3 0
r50.0_tile5 0
r50.0_tile6 0
r50.0_tile7 0
r50.0_tile8 0
r50.0_tile9 0


# last run 

- r50.0_tile6 258
- r50.0_tile7 55
- r37.0_tile2 1055
- r37.0_tile1 52